# PROYECTO FINAL

La empresa GoGreen Bikesharing, se dedica al alquiler de bicicletas y posee datos tales como la cantidad de bicis alquiladas por usuarios registrados, la cantidad de alquileres realizados por usuarios puntuales, y la cantidad total. 

A estos datos se les añadió información meteorológica, y el calendario de festivos.

Ahora buscan analizar cuáles son los aspectos que más influyen en la cantidad de bicis que van a alquilar en un día.

In [1]:
import pandas as pd

PREGUNTAS

¿en que momento del año se alquilan mas bicicletas?
Influye soleado o nubes?
Fin de semana o laborable?
los primeros y ultimos dias del mes hay mas reservas?
festivos aumenta las reservas?
la tempreatura a partir de la cual disminuye los alquileres  ->Punto ruptura
Punto maximo de alquiler
la temperatura alta influye al alquiler de espontaneos o habituales?
la humedad es un punto clave en algun momento? A partir de que punto de humedad bajan drasticamente las reservas?

fecha donde hay mas y menos reservas






In [15]:
df_bike = pd.read_csv("./bikes.csv", index_col=0)
df_bike.sample(60)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
268,269,26-09-2018,winter,0,9,0,1,1,2,26.615847,29.45210,84.8333,7.416900,684,3946,4630
86,87,28-03-2018,summer,0,3,0,1,1,1,10.838268,12.87870,30.2174,14.217668,222,1806,2028
495,496,11-05-2019,summer,1,5,0,5,1,1,21.866653,26.04165,36.0417,15.874779,1319,5711,7030
129,130,10-05-2018,summer,0,5,0,2,1,1,21.832500,26.13605,48.9167,7.749957,694,4109,4803
571,572,26-07-2019,autumn,1,7,0,4,1,1,31.843347,36.96315,59.6667,19.082471,1036,5825,6861
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
69,70,11-03-2018,spring,0,3,0,5,1,2,12.977402,15.25000,64.9565,15.608990,247,1730,1977
144,145,25-05-2018,summer,0,5,0,3,1,1,27.094153,30.77710,69.6250,10.333611,740,4238,4978
537,538,22-06-2019,autumn,1,6,0,5,1,1,31.877500,36.20605,57.3750,12.250414,964,4859,5823
586,587,10-08-2019,autumn,1,8,0,5,1,2,29.349153,33.39710,71.5833,16.000471,1065,4721,5786


In [14]:
df_bike.mnth.dtype

dtype('int64')

# limpieza string

In [10]:
df_bike.season.unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

# duplicados

In [3]:
df_bike.duplicated().sum()

0

# Nulos

In [4]:
df_bike.isnull().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

In [7]:
df_bike.shape

(730, 16)

In [6]:
df_bike.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 729
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     730 non-null    int64  
 1   dteday      730 non-null    object 
 2   season      730 non-null    object 
 3   yr          730 non-null    int64  
 4   mnth        730 non-null    int64  
 5   holiday     730 non-null    int64  
 6   weekday     730 non-null    int64  
 7   workingday  730 non-null    int64  
 8   weathersit  730 non-null    int64  
 9   temp        730 non-null    float64
 10  atemp       730 non-null    float64
 11  hum         730 non-null    float64
 12  windspeed   730 non-null    float64
 13  casual      730 non-null    int64  
 14  registered  730 non-null    int64  
 15  cnt         730 non-null    int64  
dtypes: float64(4), int64(10), object(2)
memory usage: 97.0+ KB


# definir categoricas y numericas
# .reindex las columnas a gusto o uso
# .describe
# .stb.counts()
# .stb.missing()
# pairplot - Busca relaciones - ¿Hay que hacer algun ajuste?
# Boxplot
# Outliers - Identificarlos ¿Que hacer con ellos? %de cada columna
# Heatmap con la relacion
# ajuste lineal multiple
